# Import Libraries

In [1]:
import sys
import os   
import glob
import dotenv
import argparse
dotenv.load_dotenv()

True

In [2]:
PARENT_DIR = os.path.dirname(os.getcwd())
SRC_DIR = os.path.join(PARENT_DIR, 'src')
sys.path.append(SRC_DIR)

DATA_DIR = os.path.join(PARENT_DIR, 'data')

from analyzer import CSVAnalyzer
from graph import Graph
from gpt import GPT
from CodeRetriever import CodeRetriever
from tokenizer import Tokenize

/Users/shotomorisaki/Engineering/rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Code Segment

In [3]:
files = glob.glob(os.path.join(SRC_DIR, '*'))

In [4]:
# retriever = CodeRetriever()

# # file_name = files[0]
# file_name = '../src/dummy.py'
# query = "function query"
# model = "openai" # sentence-transformers/msmarco-MiniLM-L-12-v3, openai

# retriever.process_file(file_name, query, model)

In [5]:
import warnings

# Suppress specific UserWarning about floor division
warnings.filterwarnings("ignore", message="__floordiv__ is deprecated")


In [6]:
import torch
from transformers import AutoTokenizer, CTRLLMHeadModel

tokenizer = AutoTokenizer.from_pretrained("Salesforce/ctrl")
model = CTRLLMHeadModel.from_pretrained("Salesforce/ctrl")

# CTRL was trained with control codes as the first token
inputs = tokenizer("Wikipedia The llama is", return_tensors="pt")
assert inputs["input_ids"][0, 0].item() in tokenizer.control_codes.values()

sequence_ids = model.generate(inputs["input_ids"])
sequences = tokenizer.batch_decode(sequence_ids)
sequences

outputs = model(**inputs, labels=inputs["input_ids"])
round(outputs.loss.item(), 2)

list(outputs.logits.shape)

/Users/shotomorisaki/Engineering/rag/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[1, 5, 246534]

In [ ]:
import torch
from transformers import AutoTokenizer, CTRLLMHeadModel

class TextGenerator:
    def __init__(self, model_name="Salesforce/ctrl"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = CTRLLMHeadModel.from_pretrained(model_name)

    def text_generation_pipeline(self, input_text, max_length=200):
        inputs = self.tokenizer(input_text, return_tensors="pt")
        generation_config = {"max_length": max_length}
        sequence_ids = self.model.generate(inputs["input_ids"], **generation_config)
        sequences = self.tokenizer.batch_decode(sequence_ids)
        outputs = self.model(**inputs, labels=inputs["input_ids"])
        round(outputs.loss.item(), 2)
        return list(outputs.logits.shape)
    
text_generator = TextGenerator()

english_translated_text = "The given code is a Python implementation of BLEU (Bilingual Evaluation Understudy) and smooth-BLEU, which are metrics for evaluating the quality of machine translation outputs. The code includes functions for computing n-grams, calculating BLEU scores, and applying smoothing techniques as outlined in the paper by Chin-Yew Lin and Franz Josef Och (COLING 2004). The main function `compute_bleu` takes reference and translation corpora as input and returns the BLEU score, n-gram precisions, geometric mean of n-gram precisions, and brevity penalty. Additionally, there's a helper function `_bleu` which takes a reference sentence and a translated sentence as input, and returns the BLEU score for that particular pair of sentences. The code also includes a license and citation to specify the rights and credits related to the implementation."

generated_sequences = text_generator.text_generation_pipeline(english_translated_text)
print(generated_sequences)